# Building Populations and Factions

In [10]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import sys, os, ssl
import asyncio
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")


In [11]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import app.connectors.cmdb_graph as cmdb_graph

c = cmdb_graph.CosmosdbClient()

executing local windows deployment
something wrong with your query: <class 'Exception'>


In [4]:
from app.creators import homeworld
from app.objects import species, population

Example data from the input form:

In [4]:
data = {
    "label": "form",
    "name": "worldgenform",
    "objid": "0000000000001",
    "owner": "user.username",
    "username": "user.username",
    "accountid": "0000000000001",
    "conformity": 0.1,
    "constitution": 0.5,
    "literacy": 0.5,
    "aggression": 0.5,
    "num_planets": 4,
    "num_moons": 10,
    "starting_pop": 30,
    "organics": 0.5,
    "minerals": 0.5,
}

In [7]:
nodes, edges = homeworld.build_people(data)
nodes

'nodes'

In [6]:
[n for n in nodes if n["label"] == "faction"]

TypeError: string indices must be integers

In [7]:
0 in [[0, 0]]

False

In [8]:
options = [[1, 1], [-1, 1], [1, -1], [-1, -1]]
pick = options[np.random.choice([0, 1, 2, 3])]
pick

[1, 1]

In [9]:
new_pick = options[np.random.choice([0, 1, 2, 3])]
new_pick

[1, -1]

In [10]:
all_pick = np.add(pick, new_pick)
all_pick.tolist()

[2, 0]

# Getting resources and populations of factions

In [ ]:
objid = "0293168419375"
pops_query = f"g.V().has('objid','{objid}').in('isIn').valueMap()"
resources_query = f"g.V().has('objid','{objid}').out('has').valueMap()"
c.add_query(pops_query)
c.add_query(resources_query)
c.run_queries()

In [14]:
c.res[pops_query]

[{'name': ['Snorelau Terdeciay'],
  'objid': ['1054627691861'],
  'conformity': ['0.647'],
  'literacy': ['0.75'],
  'aggression': ['0.389'],
  'constitution': ['0.419'],
  'health': ['0.7'],
  'isIn': ['0293168419375'],
  'industry': ['0.404'],
  'wealth': ['0.577'],
  'factionLoyalty': ['0.712'],
  'isIdle': ['true'],
  'userguid': ['8d5b667f-b225-4641-b499-73b77558ff86'],
  'objtype': ['pop']},
 {'isIdle': ['true'],
  'name': ['Snorelau Hinos'],
  'objid': ['1059544263233'],
  'conformity': ['0.733'],
  'literacy': ['0.79'],
  'aggression': ['0.263'],
  'constitution': ['0.429'],
  'health': ['0.7'],
  'isIn': ['0293168419375'],
  'industry': ['0.346'],
  'wealth': ['0.5680000000000001'],
  'factionLoyalty': ['0.686'],
  'userguid': ['8d5b667f-b225-4641-b499-73b77558ff86'],
  'objtype': ['pop']},
 {'name': ['Snorelau Canis'],
  'objid': ['5073166215030'],
  'conformity': ['0.659'],
  'literacy': ['0.605'],
  'aggression': ['0.32'],
  'constitution': ['0.431'],
  'health': ['0.7'],
 

In [20]:
query = f"g.V().has('objid','{objid}').out().valueMap()"
c.run_query(query)
c.res

[]

In [21]:
pd.DataFrame(c.clean_nodes(c.res))

""
